# Quick start

## Setup Virtual Environment

```sh
uv venv -p 3.12
uv init
```

### Activate the virtual environment

::: {.panel-tabset}
#### Windows

```sh
.venv\Scripts\activate.bat
```

#### MacOS / Linux

```sh
source .venv/bin/activate
```
:::


## API Key Setup

Think of an API Key as a **hotel key card**.

* **The Hotel (Server):** Has resources (rooms) but keeps them locked.
* **The Guest (Client):** Wants access.
* **The Key Card (API Key):** Identifies you and proves you are allowed to enter specific rooms.

---

### What & Why

An API key is a unique string of characters used to identify the calling program.

* **Identification:** Keys "authenticate the calling project," allowing the server to recognize who is asking for data.
* **Control:** This lets the server track usage for billing and enforce limits (quotas) so one user doesn't crash the system.

---

### Security Risks

If you lose your key, it is like dropping your credit card.

* **Theft:** Attackers can use your key to make requests on your behalf.
* **Consequences:** You suffer **financial loss** (paying for their usage) or **service denial** (they use up your available quota).

> **Rule:** Never post keys on public sites like GitHub.

### How to Set Your API Key?

This project uses OpenRouter (**The Unified Interface For LLMs**), via LiteLLM to access the DeepSeek model, which requires an API key. If you don't already have an OpenRouter API key, you can create one for free at: [OpenRouter](https://openrouter.ai/keys).

Write your API key into an `.env` file as an environment variable, as follows:

```sh
OPENROUTER_API_KEY=...
```

> Note: make sure to add it to `.gitignore` to avoid committing it to the repository.
> 
> Note: this is different than the `.venv` file used for the virtual environment.

In [ ]:
import os
from typing import Literal

from dotenv import load_dotenv

In [ ]:
load_dotenv()

# We use OpenRouter for the agent — set OPENROUTER_API_KEY in .env
# Get your key at https://openrouter.ai/keys
if not os.environ.get("OPENROUTER_API_KEY"):
    raise RuntimeError(
        "OPENROUTER_API_KEY is not set. Add it to your .env file, e.g.:\n"
        "OPENROUTER_API_KEY=your-openrouter-api-key"
    )

## Tool

Example using ddgs (duckduckgo-search) package for web search

```sh
uv add ddgs
```

In [ ]:
from ddgs import DDGS

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
search.invoke("Obama's first name?")

Impersonate 'chrome_109' does not exist, using 'random'


"قبل ٤ أيام · Michelle LaVaughn Robinson Obama (née Robinson; born January 17, 1964) is an American attorney and author who served as First Lady of the United States from ... قبل يوم واحد · Barack Obama (@BarackObama) - Posts - Dad, husband, President, citizen. | X (formerly Twitter) ٠١/٠٢/٢٠٢٦ · Barack Obama. 56795995 likes · 2140424 talking about this. Dad, husband, former President, citizen. قبل يومين · 42M followers · 10 following · 1157 posts · @barackobama: “Dad, husband, President, citizen.” قبل ٥ ساعات · Yeah he has. It's Obama. And by that I mean it's also his first. And his middle name. His name is Obama Obama Obama."

In [8]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(output_format="list", backend="news")

search.invoke("Obama")

[{'snippet': 'Former President Barack Obama admitted that there are aliens. Brian Tyler Cohen, a liberal...',
  'title': "Obama Confirms Existence of Aliens, But Says They Aren't Where People Think",
  'link': 'https://www.huffpost.com/entry/obama-says-aliens-are-real-area-51_n_69920b94e4b049774f81a127',
  'date': '2026-02-16T07:36:35+00:00',
  'source': 'The Huffington Post'},
 {'snippet': 'Barack Obama and Michelle Obama attended the NBA All-Star Game with one special person in tow: Their...',
  'title': 'Sasha Obama Makes Rare Appearance With Parents Barack, Michelle Obama at NBA...',
  'link': 'https://www.eonline.com/videos/2487507011757/sasha-obama-makes-rare-appearance-with-parents-barack-michelle-obama-at-nba-all-star-game',
  'date': '2026-02-16T20:36:35+00:00',
  'source': 'E! Online'},
 {'snippet': 'Former US President Barack Obama said he’s seen no evidence that aliens have made contact with Earth...',
  'title': 'Obama clarifies alien comments after telling podcast ‘they’r

In [ ]:
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search using DuckDuckGo (via ddgs package)"""
    # The topic can be appended to the query to bias results
    q = f"{query} ({topic})" if topic != "general" else query
    results = []
    with DDGS() as ddgs:
        search_gen = ddgs.text(q)
        for i, result in enumerate(search_gen):
            if i >= max_results:
                break
            res = {
                "title": result.get("title"),
                "url": result.get("href"),
            }
            if include_raw_content:
                res["snippet"] = result.get("body")
            results.append(res)
    return results

## Agent

In [ ]:
from deepagents import create_deep_agent
from langchain_openai import ChatOpenAI

### Prompt

In [ ]:
# System prompt to steer the agent to be an expert researcher
research_instructions = """You are an expert researcher. Your job is to conduct thorough research and then write a polished report.

You have access to an internet search tool as your primary means of gathering information.

## `internet_search`

Use this to run an internet search for a given query. You can specify the max number of results to return, the topic, and whether raw content should be included.
"""

### LLM (model)

In [ ]:
# OpenRouter: OpenAI-compatible API — https://openrouter.ai/
openrouter_model = ChatOpenAI(
    model="openai/gpt-5-nano",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)

### Initialize and Run

In [ ]:
agent = create_deep_agent(
    model=openrouter_model,
    tools=[internet_search],
    system_prompt=research_instructions
)

In [ ]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What is langgraph?"
            }
        ]
    }
)

In [ ]:
# Print the agent's response
print(result["messages"][-1].content)

LangGraph is a framework for building and orchestrating resilient, stateful AI agents. It’s part of the LangChain ecosystem and provides graph-based coordination for multiple agents or tasks, enabling more reliable, long-running workflows than a single-agent approach.

Key ideas and features:
- Graph-based orchestration: you define a workflow as a graph of tasks/agents with dependencies, so complex multi-step processes can be composed and managed.
- Stateful / memory-enabled: agents can maintain state across steps, which is useful for long conversations or multi-turn tasks.
- Fault tolerance and retries: designed to handle failures gracefully, with mechanisms to retry or route around errors.
- Modularity and reuse: components (agents, tools, memory, policies) can be composed and reused across different workflows.
- Suitable for long-running or enterprise-grade automation: ideal for pipelines, complex decision processes, or multi-agent collaboration scenarios.

Typical uses:
- Building 